## Visualization #3

    Is the distribution of students in NCWIT data set similar to the population for ACT scores?

In [242]:
import pandas as pd
import bokeh
import numpy as np

from bokeh.io import show, output_notebook, push_notebook
from bokeh.plotting import figure
from bokeh.layouts import gridplot, widgetbox, layout
from bokeh.models import ColumnDataSource
from ipywidgets import interact
from scipy.stats import norm

output_notebook()
pd.options.display.max_rows = 50
pd.options.display.max_columns = 999

Loading BokehJS ...

In [243]:
ncwit_df = pd.read_csv("NCWIT-Data Cleaned.csv", low_memory = False)
# Left this out so code below only calculates reported values.
#ncwit_df = ncwit_df.fillna(0)
ncwit_df.head(30)

,Record#,CIP# Only,Major Program Name,Degree Level,NCWIT Participant,Institution,What degrees does your institution offer?,School Year,When do students typically declare their major?,"Enroll, Female: Avg. ACT Math Score (Enrl F)","Enroll, Female: Asian (Enrl F)","Enroll, Female: Average Age (Enrl F)","Enroll, Female: Average HS GPA (Enrl F)","Enroll, Female: Avg. SAT Math Score (Enrl F)","Enroll, Female: Black/African American (Enrl F)","Enroll, Female: Hispanics of any race (Enrl F)","Enroll, Female: American Indian/Alaska Native (Enrl F)","Enroll, Female: Native Hawaiian/Other Pacific Islander (Enrl F)","Enroll, Female: Two or more races (Enrl F)","Enroll, Female: New Enrollments (Enrl F)","Enroll, Female: Transfer Students (Enrl F)","Enroll, Female: US Citizens (Enrl F)","Enroll, Female: White (Enrl F)","Enroll, Male: Avg. ACT Math Score (Enrl M)","Enroll, Male: Asian (Enrl M)","Enroll, Male: Average Age (Enrl M)","Enroll, Male: Average HS GPA (Enrl M)","Enroll, Male: Avg. SAT Math Score (Enrl M)","Enroll, Male: Black/African American (Enrl M)","Enroll, Male: Hispanics of any race (Enrl M)","Enroll, Male: American Indian/Alaska Native (Enrl M)","Enroll, Male: Native Hawaiian/Other Pacific Islander (Enrl M)","Enroll, Male: Two or more races (Enrl M)","Enroll, Male: New Enrollments (Enrl M)","Enroll, Male: Transfer Students (Enrl M)","Enroll, Male: US Citizens (Enrl M)","Enroll, Male: White (Enrl M)","Freshmen, Female: Asian (Fshm F)","Freshmen, Female: Black/African American (Fshm F)","Freshmen, Female: Cumulative GPA (Fshm F)","Freshmen, Female: Enrolled in DIFFERENT MAJOR (Fshm F)","Freshmen, Female: Enrolled in SAME MAJOR (Fshm F)","Freshmen, Female: Graduated (Fshm F)","Freshmen, Female: Hispanics of any race (Fshm F)","Freshmen, Female: Left Institution (not graduated) (Fshm F)","Freshmen, Female: American Indian/Alaska Native (Fshm F)","Freshmen, Female: Native Hawaiian/Other Pacific Islander (Fshm F)","Freshmen, Female: Two or more races (Fshm F)","Freshmen, Female: Total Declared Majors (Fshm F)","Freshmen, Female: US Citizens (Fshm F)","Freshmen, Female: White (Fshm F)","Freshmen, Male: Asian (Fshm M)","Freshmen, Male: Black/African American (Fshm M)","Freshmen, Male: Cumulative GPA (Fshm M)","Freshmen, Male: Enrolled in DIFFERENT MAJOR (Fshm M)","Freshmen, Male: Enrolled in SAME MAJOR (Fshm M)","Freshmen, Male: Graduated (Fshm M)","Freshmen, Male: Hispanics of any race (Fshm M)","Freshmen, Male: Left Institution (not graduated) (Fshm M)","Freshmen, Male: American Indian/Alaska Native (Fshm M)","Freshmen, Male: Native Hawaiian/Other Pacific Islander (Fshm M)","Freshmen, Male: Two or more races (Fshm M)","Freshmen, Male: Total Declared Majors (Fshm M)","Freshmen, Male: US Citizens (Fshm M)","Freshmen, Male: White (Fshm M)","Sophomores, Female: Asian (Sph. F)","Sophomores, Female: Black/African American (Sph. F)","Sophomores, Female: Cumulative GPA (Sph. F)","Sophomores, Female: Enrolled in DIFFERENT MAJOR (Sph. F)","Sophomores, Female: Enrolled in SAME MAJOR (Sph. F)","Sophomores, Female: Graduated (Sph. F)","Sophomores, Female: Hispanics of any race (Sph. F)","Sophomores, Female: Left Institution (not graduated) (Sph. F)","Sophomores, Female: American Indian/Alaska Native (Sph. F)","Sophomores, Female: Native Hawaiian/Other Pacific Islander (Sph. F)","Sophomores, Female: Two or more races (Sph. F)","Sophomores, Female: Total Declared Majors (Sph. F)","Sophomores, Female: US Citizens (Sph. F)","Sophomores, Female: White (Sph. F)","Sophomores, Male: Asian (Sph. M)","Sophomores, Male: Black/African American (Sph. M)","Sophomores, Male: Cumulative GPA (Sph. M)","Sophomores, Male: Enrolled in DIFFERENT MAJOR (Sph. M)","Sophomores, Male: Enrolled in SAME MAJOR (Sph. M)","Sophomores, Male: Graduated (Sph. M)","Sophomores, Male: Hispanics of any race (Sph. M)","Sophomores, Male: Left Institution (not graduated) (Sph. M)","Sophomores, Male: American Indian/Alaska Native (Sph. M)","Sophomores, Male: Native Hawaiian/Other Paci

In [244]:
ACT_df = pd.read_csv("ACT Data.csv")
ACT_df = ACT_df.loc[ACT_df.Year > 2005]
#fMean = 21.0
#mMean = 21.2
#stdDev = 5.0
#headerYear = "2007"
#print(ACT_df.head())
def update(Year):
    headerYear = Year
    ncwitYear = str(Year)+"-"+str(int(Year)+1)
    print(ncwitYear)
    ncwitMod = ncwit_df.loc[ncwit_df["School Year"] == ncwitYear].copy()
    actMod = ACT_df.loc[ACT_df["Year"] == int(Year)].copy()
    fMean = actMod["Female"]
    mMean = actMod["Male"]
    stdDev = actMod["Standard Deviation"]
    fVal, fCounts = np.unique(ncwitMod["Enroll, Female: Avg. ACT Math Score (Enrl F)"].round(0).dropna(), return_counts=True)
    fCounts = fCounts/np.sum(fCounts)
    print("fMean", fMean)

    mVal, mCounts = np.unique(ncwitMod["Enroll, Male: Avg. ACT Math Score (Enrl M)"].round(0).dropna(), return_counts=True)
    mCounts = mCounts/np.sum(mCounts)
    #print(mVal,mCounts)
    fNorm.data_source.data["fMean"] = ColumnDataSource.from_df(fMean)
    fNorm.data_source.data["stdDev"] = stdDev
    fBins.data_source.data["fVal"] = fVal
    fBins.data_source.data["fCounts"] = fCounts
    #p.data_source.data["headerYear"] = headerYear
    mNorm.data_source.data["mMean"] = mMean
    mNorm.data_source.data["stdDev"] = stdDev
    mBins.data_source.data["mVal"] = mVal
    mBins.data_source.data["mCounts"] = mCounts
    #q.data_source.data["headerYear"] = headerYear
    push_notebook()
# ncwit2016.head()


In [245]:
# https://stackoverflow.com/questions/12207326/frequency-table-for-a-single-variable
# Code finds the # of unique values for ACT scores (after rounding and dropping nan values) and
# returns the count of each observation of each value

# New figure for Female data.
p = figure(x_range= [0,36.5], plot_height = 500, title = headerYear+" Average Score Distribution - Female")
fBins = p.vbar(x=fVal, top = fCounts, width = 0.9)
p.y_range.start = 0
# Generate a line for the distribution of scores that year for Females
x = np.linspace(0,36,500)
fNorm = p.line(x, norm.pdf(x, fMean, stdDev), line_width=2, color = "red")

# New figure for Male data.
q = figure(x_range= [0,36.5], plot_height = 500, title = headerYear+" Average Score Distribution - Male")
mBins = q.vbar(x=mVal, top = mCounts, width = 0.9)
q.y_range.start = 0
# Generate a line for the distribution of scores that year for Males
mNorm = q.line(x, norm.pdf(x, mMean, stdDev), line_width=2, color = "red")

grid = layout([p], [q])

show(grid, notebook_handle = True)

In [246]:
interact(update, Year=["2007","2008","2009","2010","2011","2012","2013","2014","2015","2016"])

<function __main__.update>